In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

from microhhpy.chem import calc_tuv_photolysis
from microhhpy.utils import get_data_file

Generating photolysis rates for the KPP chemistry scheme is somewhat cumbersome. Currently, we need to generate input files for TUV and then call it from Python. Since TUV is written in Fortran, it is not straightforward to include it in a Python package as a pre-compiled executable. As a result, the process requires several extra steps:

1. Download the TUV source code from: https://www2.acom.ucar.edu/modeling/tuv-download . This tutorial uses TUV 5.4 from May 2021.
2. Unpack and compile the code with `make`. This only requires the `gfortran` compiler.
3. Use the path to the TUV root (e.g. `/path/to/TUV5.4`) in the `calc_tuv_photolysis()` call below.

In [ ]:
# Path the the TUV directory containing the compiled `tuv` binary:
tuv_path = '/home/bart/meteo/models/microhhpy/external/TUV/V5.4'

# Case name. This must be exactly 6 characters long. Not 5, not 7, because flexibility is overrated…
name = 'microh'

# The TUV routines in `microhhpy` works from a base input file which is included in the repository.
# Our wrapper only updates the date/time and lat/lon parameters. If you need to change more, simply
# copy this file and edit it before passing the path to `calc_tuv_photolysis` below.
input_file = get_data_file('microhh_tuv.base')

In [ ]:
!cat {input_file}

In [ ]:
# TUV can only generated output for a single day. More days are possible
# with out Python wrapper, which simply calls the TUV code multiple times.
start_date = datetime(year=2020, month=9, day=1, hour=12)
end_date = datetime(year=2020, month=9, day=3, hour=18)

lon = 5.1
lat = 50.1

# Option to suppress the print statements from TUV:
suppress_stdout=True

tuv_df = calc_tuv_photolysis(
        input_file,
        tuv_path,
        name,
        start_date,
        end_date,
        lon,
        lat,
        suppress_stdout)

In [ ]:
plt.figure()
plt.plot(tuv_df.index, tuv_df.jno2)
plt.xlabel('Date')
plt.ylabel(r'J$_\mathrm{NO_2}$ (s$^{-1}$)')